In [1]:
import os
os.chdir("/root/data/DBP_sa_bc/")
from os.path import join as pj
import argparse
import sys
sys.path.append("modules")
import utils
import numpy as np
import scib
import scib.metrics as me
import anndata as ad
import scipy
import pandas as pd
import re

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--task', type=str, default='wnn_rna')
parser.add_argument('--experiment', type=str, default='e1')
parser.add_argument('--model', type=str, default='default')
parser.add_argument('--init_model', type=str, default='sp_00001899')
parser.add_argument('--method', type=str, default='harmony')
parser.add_argument('--K', type=str, default='20')
# parser.add_argument('--method', type=str, default='midas_embed')
o, _ = parser.parse_known_args()  # for python interactive
# o = parser.parse_args()

In [3]:
if "DBP_sa_bc" in o.method:
    result_dir = pj("result", "comparison", o.task, o.method, o.experiment, o.init_model)
else:
    result_dir = pj("result", "comparison", o.task, o.method, o.K)
cfg_task = re.sub("_atlas|_generalize|_transfer|_ref_.*", "", o.task)
data_config = utils.load_toml("configs/data.toml")[cfg_task]
for k, v in data_config.items():
    vars(o)[k] = v
model_config = utils.load_toml("configs/model.toml")["default"]
if o.model != "default":
    model_config.update(utils.load_toml("configs/model.toml")[o.model])
for k, v in model_config.items():
    vars(o)[k] = v
o.s_joint, o.combs, *_ = utils.gen_all_batch_ids(o.s_joint, o.combs)

In [4]:
# Load cell type labels
if o.task == "hae":
    labels = []
    for raw_data_dir in o.raw_data_dirs:
        label = utils.load_csv(pj(raw_data_dir, "label", "labels.csv"))
        labels += utils.transpose_list(label)[13][1:]
    labels = np.array(labels)
    print(np.unique(labels))
elif o.task == "wnn_rna":
    labels = []
    for raw_data_dir in o.raw_data_dirs:
        label = utils.load_csv(pj(raw_data_dir, "label", "meta.csv"))
        labels += utils.transpose_list(label)[10][1:]
    labels = np.array(labels)
    print(np.unique(labels))
elif o.task == "lung_ts":
    labels = []
    for raw_data_dir in o.raw_data_dirs:
        label = utils.load_csv(pj(raw_data_dir, "label", "meta.csv"))
        labels += utils.transpose_list(label)[13][1:]
    labels = np.array(labels)
    print(np.unique(labels))
elif o.task == "wnn_rnas7":
    labels = []
    for raw_data_dir in o.raw_data_dirs:
        label = utils.load_csv(pj(raw_data_dir, "label", "meta.csv"))
        labels += utils.transpose_list(label)[10][1:]
    labels = np.array(labels)
    print(np.unique(labels))
    # labels = []
    # for raw_data_dir in o.raw_data_dirs:
    #     label = utils.load_csv(pj(raw_data_dir, "label_seurat", "l1.csv"))
    #     labels += utils.transpose_list(label)[1][1:]
    # labels = np.array(labels)
    # print(np.unique(labels))

['B' 'CD4 T' 'CD8 T' 'DC' 'Mono' 'NK' 'other' 'other T']


In [5]:
# Load predicted latent variables
o.mods = ["rna"]
o.pred_dir = pj("result", o.task, o.experiment, o.model, "predict", o.init_model)
pred = utils.load_predicted(o)

Loading predicted variables ...
Loading subset 0: z, joint


100%|██████████| 25/25 [00:00<00:00, 47.96it/s]


Loading subset 0: w, joint


100%|██████████| 25/25 [00:00<00:00, 87.37it/s]


Loading subset 0: A, joint


...

Loading subset 7: w, joint


100%|██████████| 35/35 [00:00<00:00, 45.64it/s]


Loading subset 7: A, joint


100%|██████████| 1/1 [00:00<00:00,  3.55it/s]


Converting to numpy ...
Converting subset 0: s, joint
Converting subset 0: z, joint
Converting subset 0: w, joint
Converting subset 0: A, joint
Converting subset 1: s, joint
Converting subset 1: z, joint
Converting subset 1: w, joint
Converting subset 1: A, joint
Converting subset 2: s, joint
Converting subset 2: z, joint
Converting subset 2: w, joint
Converting subset 2: A, joint
Converting subset 3: s, joint
Converting subset 3: z, joint
Converting subset 3: w, joint
Converting subset 3: A, joint
Converting subset 4: s, joint
Converting subset 4: z, joint
Converting subset 4: w, joint
Converting subset 4: A, joint
Converting subset 5: s, joint
Converting subset 5: z, joint
Converting subset 5: w, joint
Converting subset 5: A, joint
Converting subset 6: s, joint
Converting subset 6: z, joint
Converting subset 6: w, joint
Converting subset 6: A, joint
Converting subset 7: s, joint
Converting subset 7: z, joint
Converting subset 7: w, joint
Converting subset 7: A, joint


In [6]:
if o.method in ["DBP_sa_bc", "mofa", "harmony", "liger", "scETM", "scanorama_embed", "seurat_rpca", "seurat_cca", "pca", "LDVAE"]:
    output_type = "embed"
elif o.method in [
    "midas_feat+wnn", 
    "harmony+wnn", 
    "pca+wnn",
    "seurat_cca+wnn",
    "seurat_rpca+wnn",
    "scanorama_embed+wnn",
    "scanorama_feat+wnn",
    "liger+wnn",
    "bbknn",
    ]:
    output_type = "graph"
else:
    assert False, o.method+": invalid method!"

In [7]:
embed = "X_emb"
batch_key = "batch"
label_key = "label"
cluster_key = "cluster"
si_metric = "euclidean"
subsample = 0.5
verbose = False

In [8]:
w = pred["w"]["joint"]
c = pred["z"]["joint"][:, :o.dim_c]*w
s = pred["s"]["joint"]

if o.method == "DBP_sa_bc":
    adata = ad.AnnData(c)
    adata.obsm[embed] = c
    adata.obs[batch_key] = s.astype(str)
    adata.obs[batch_key] = adata.obs[batch_key].astype("category")
    adata.obs[label_key] = labels
    adata.obs[label_key] = adata.obs[label_key].astype("category")
elif o.method in ["mofa", "harmony", "liger", "scanorama_embed", "seurat_rpca", "seurat_cca", "pca"]:
    adata = ad.AnnData(c*0)
    embeddings = utils.load_csv(pj(result_dir, "embeddings.csv"))
    adata.obsm[embed] = np.array(embeddings)[1:, 1:].astype(np.float32)
    adata.obs[batch_key] = s.astype(str)
    adata.obs[batch_key] = adata.obs[batch_key].astype("category")
    adata.obs[label_key] = labels
    adata.obs[label_key] = adata.obs[label_key].astype("category")
elif o.method in ["scETM", "LDVAE"]:
    adata = ad.AnnData(c*0)
    embeddings = utils.load_csv(pj(result_dir, "embeddings.csv"))
    adata.obsm[embed] = np.array(embeddings).astype(np.float32)
    adata.obs[batch_key] = s.astype(str)
    adata.obs[batch_key] = adata.obs[batch_key].astype("category")
    adata.obs[label_key] = labels
    adata.obs[label_key] = adata.obs[label_key].astype("category")
elif o.method in [
    "midas_feat+wnn", 
    "harmony+wnn", 
    "pca+wnn",
    "seurat_cca+wnn",
    "seurat_rpca+wnn",
    "scanorama_embed+wnn",
    "scanorama_feat+wnn",
    "liger+wnn",
    "bbknn",
    ]:
    adata = ad.AnnData(c*0)
    adata.obs[batch_key] = s.astype(str)
    adata.obs[batch_key] = adata.obs[batch_key].astype("category")
    adata.obs[label_key] = labels
    adata.obs[label_key] = adata.obs[label_key].astype("category")
    adata.obsp["connectivities"] = scipy.io.mmread(pj(result_dir, "connectivities.mtx")).tocsr()
    adata.uns["neighbors"] = {'connectivities_key': 'connectivities'}

In [9]:
results = {}

print('clustering...')
res_max, nmi_max, nmi_all = scib.clustering.opt_louvain(adata, label_key=label_key,
    cluster_key=cluster_key, function=me.nmi, use_rep=embed, verbose=verbose, inplace=True)

results['graph_conn'] = me.graph_connectivity(adata, label_key=label_key)
print("graph_conn: " + str(results['graph_conn']))

results['batch_ASW'] = me.silhouette_batch(adata, label_key=label_key, batch_key=batch_key,
    embed=embed, verbose=verbose)
print("batch_ASW: " + str(results['batch_ASW']))

type_ = "knn" if output_type == "graph" else None
results['kBET'] = me.kBET(adata, batch_key=batch_key, label_key=label_key, embed=embed, 
    type_=type_, verbose=verbose)
print("kBET: " + str(results['kBET']))

# results['iLISI'] = me.ilisi_graph(adata, batch_key=batch_key, type_="knn",
#     subsample=subsample*100, n_cores=1, verbose=verbose)
# print("iLISI: " + str(results['iLISI']))
results['iLISI'] = me.ilisi_graph(adata, batch_key=batch_key, type_="knn",
    subsample=subsample*100, verbose=verbose)
print("iLISI: " + str(results['iLISI']))


results['label_ASW'] = me.silhouette(adata, label_key=label_key, embed=embed)
print("label_ASW: " + str(results['label_ASW']))

results['il_score_ASW'] = me.isolated_labels(adata, label_key=label_key, batch_key=batch_key,
    embed=embed, cluster=False, verbose=verbose)
print("il_score_ASW: " + str(results['il_score_ASW']))

results['il_score_f1'] = me.isolated_labels(adata, label_key=label_key, batch_key=batch_key,
    embed=embed, cluster=True, verbose=verbose)
print("il_score_f1: " + str(results['il_score_f1']))

results['NMI'] = me.nmi(adata, group1=cluster_key, group2=label_key, method='arithmetic')
print("NMI: " + str(results['NMI']))

results['ARI'] = me.ari(adata, group1=cluster_key, group2=label_key)
print("ARI: " + str(results['ARI']))

results['cLISI'] = me.clisi_graph(adata, batch_key=batch_key, label_key=label_key, type_="knn",
    subsample=subsample*100,  verbose=verbose)
print("cLISI: " + str(results['cLISI']))

# results = {k: float(v) for k, v in results.items()}
# results['batch_score'] = np.nanmean([results['graph_conn'], results['batch_ASW'], results['kBET'], results['iLISI']])
# results['bio_score'] = np.nanmean([results['label_ASW'], results['il_score_ASW'], results['il_score_f1'], 
#                                    results['NMI'], results['ARI'], results['cLISI']])
# results["overall_score"] = float(0.4 * results['batch_score'] + 0.6 * results['bio_score'])

df = pd.DataFrame({
    'graph_conn':     [results['graph_conn']],
    'batch_ASW':      [results['batch_ASW']],
    'kBET':           [results['kBET']],
    'iLISI':          [results['iLISI']],
    # 'batch_score':    [results['batch_score']],
    'label_ASW':      [results['label_ASW']],
    'il_score_ASW':   [results['il_score_ASW']],
    'il_score_f1':    [results['il_score_f1']],   
    'NMI':            [results['NMI']],
    'ARI':            [results['ARI']],
    'cLISI':          [results['cLISI']],
    # 'bio_score':      [results['bio_score']],
    # 'overall_score':  [results['overall_score']]
})
print(df)
utils.mkdirs(result_dir, remove_old=False)
df.to_excel(pj(result_dir, "metrics_batch_bio.xlsx"), index=False)

clustering...


/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated function (or staticmethod) opt_louvain.
  """


graph_conn: 0.9514722402564708
batch_ASW: 0.9008520102784057
Adding diffusion to step 4


/root/anaconda3/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:264: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  category=DeprecationWarning)
/root/anaconda3/lib/python3.7/site-packages/rpy2/robjects/numpy2ri.py:188: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  category=DeprecationWarning)


Adding diffusion to step 4
Adding diffusion to step 4
Adding diffusion to step 5
Adding diffusion to step 4
Adding diffusion to step 4
kBET: 0.2698172021895944
iLISI: 0.5061031894126949
label_ASW: 0.6529103815555573
il_score_ASW: 0.5875662091712002
Compute neighbors on rep X_emb
Compute neighbors on rep X_emb
Compute neighbors on rep X_emb
Compute neighbors on rep X_emb
Compute neighbors on rep X_emb
Compute neighbors on rep X_emb
Compute neighbors on rep X_emb


In [ ]:
# results = {}

# print('clustering...')
# res_max, nmi_max, nmi_all = scib.clustering.opt_louvain(adata, label_key=label_key,
#     cluster_key=cluster_key, function=me.nmi, use_rep=embed, verbose=verbose, inplace=True)

# results['NMI'] = me.nmi(adata, group1=cluster_key, group2=label_key, method='arithmetic')
# print("NMI: " + str(results['NMI']))

# results['ARI'] = me.ari(adata, group1=cluster_key, group2=label_key)
# print("ARI: " + str(results['ARI']))

# type_ = "knn" if output_type == "graph" else None
# results['kBET'] = me.kBET(adata, batch_key=batch_key, label_key=label_key, embed=embed, 
#     type_=type_, verbose=verbose)
# print("kBET: " + str(results['kBET']))

# results['il_score_f1'] = me.isolated_labels(adata, label_key=label_key, batch_key=batch_key,
#     embed=embed, cluster=True, verbose=verbose)
# print("il_score_f1: " + str(results['il_score_f1']))

# results['graph_conn'] = me.graph_connectivity(adata, label_key=label_key)
# print("graph_conn: " + str(results['graph_conn']))

# # results['cLISI'] = me.clisi_graph(adata, batch_key=batch_key, label_key=label_key, type_="knn",
# #     subsample=subsample*100, n_cores=1, verbose=verbose)
# # print("cLISI: " + str(results['cLISI']))

# results['cLISI'] = me.clisi_graph(adata, batch_key=batch_key, label_key=label_key, type_="knn",
#     subsample=subsample*100,  verbose=verbose)
# print("cLISI: " + str(results['cLISI']))

# # results['iLISI'] = me.ilisi_graph(adata, batch_key=batch_key, type_="knn",
# #     subsample=subsample*100, n_cores=1, verbose=verbose)
# # print("iLISI: " + str(results['iLISI']))
# results['iLISI'] = me.ilisi_graph(adata, batch_key=batch_key, type_="knn",
#     subsample=subsample*100, verbose=verbose)
# print("iLISI: " + str(results['iLISI']))

# results = {k: float(v) for k, v in results.items()}
# results['batch_score'] = np.nanmean([results['iLISI'], results['graph_conn'], results['kBET']])
# results['bio_score'] = np.nanmean([results['NMI'], results['ARI'], results['il_score_f1'], results['cLISI']])
# results["overall_score"] = float(0.4 * results['batch_score'] + 0.6 * results['bio_score'])

# df = pd.DataFrame({
#     'iLISI':          [results['iLISI']],
#     'graph_conn':     [results['graph_conn']],
#     'kBET':           [results['kBET']],
#     'batch_score':    [results['batch_score']],
#     'NMI':            [results['NMI']],
#     'ARI':            [results['ARI']],
#     'il_score_f1':    [results['il_score_f1']],
#     'cLISI':          [results['cLISI']],
#     'bio_score':      [results['bio_score']],
#     'overall_score':  [results['overall_score']]
# })
# print(df)
# utils.mkdirs(result_dir, remove_old=False)
# df.to_excel(pj(result_dir, "metrics_batch_bio.xlsx"), index=False)